In [ ]:
!pip install hazm
!pip install transformers

In [10]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import hazm
import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer

In [5]:
data = pd.read_csv('/kaggle/input/snappfood-persian-sentiment-analysis/Snappfood - Sentiment Analysis.csv', on_bad_lines='skip' , delimiter='\t')
data = data[['comment', 'label_id']]
data.head()

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0.0
4,شیرینی وانیلی فقط یک مدل بود.,0.0


In [6]:
data.dropna(inplace=True)
data = data.drop_duplicates(subset=['comment'], keep='first')
data = data.reset_index(drop=True)
data['label_id'] = data['label_id'].astype(int)
labels = ['HAPPY', 'SAD']
data['comment_len_by_words'] = data['comment'].apply(lambda t: len(hazm.word_tokenize(t)))

data.head(20)

,comment,label_id,comment_len_by_words
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1,9
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0,28
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1,19
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0,21
4,شیرینی وانیلی فقط یک مدل بود.,0,7
5,بدترین پیتزایی که تا به حال خورده بودم,1,7
6,از همه لحاظ عالی ممنونم,0,5
7,کیفیت غذا متوسط رو به پایین بود انگار داخل یه ...,1,25
8,همه اقلام تازه و به روز وخیلیییییی سریع بدستم ...,0,12
9,همه چی خوب ولی هات داگ دورش کلا سوخته بود و دا...,1,14


In [7]:
minlim, maxlim = 3, 62

data['comment_len_by_words'] = data['comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
data = data.dropna(subset=['comment_len_by_words'])
data = data.reset_index(drop=True)

In [8]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=data['comment_len_by_words']
))

fig.update_layout(
    title_text='Distribution of word counts within comments',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [11]:
data['comment'] = data['comment'].apply(lambda x: re.sub(r'[\da-zA-Z\!\(\)\-\[\]\{\}\;\:\'\"\\\,\<\>\.\/\?\@\#\$\%\^\&\*\_\~\؟\،\٪\×\÷\»\«]', '', x))
normalizer = hazm.Normalizer()
data['comment'] = data['comment'].apply(lambda x: normalizer.normalize(x.strip()))
data['cleaned_comment_len_by_words'] = data['comment'].apply(lambda t: len(hazm.word_tokenize(t)))
data['cleaned_comment_len_by_words'] = data['cleaned_comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
data = data.dropna(subset=['cleaned_comment_len_by_words'])
data = data.reset_index(drop=True)

data.head(20)

,comment,label_id,comment_len_by_words,cleaned_comment_len_by_words
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1,9.0,9.0
1,قرار بود ساعته برسه ولی نیم ساعت زودتر از موقع...,0,28.0,25.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره فقط ...,1,19.0,17.0
3,عالی بود همه چه درست و به اندازه و کیفیت خوب ا...,0,21.0,20.0
4,شیرینی وانیلی فقط یک مدل بود,0,7.0,6.0
5,بدترین پیتزایی که تا به حال خورده بودم,1,7.0,7.0
6,از همه لحاظ عالی ممنونم,0,5.0,5.0
7,کیفیت غذا متوسط رو به پایین بود انگار داخل یه ...,1,25.0,22.0
8,همه اقلام تازه و به‌روز وخیلیی سریع بدستم رسید...,0,12.0,11.0
9,همه چی خوب ولی هات داگ دورش کلا سوخته بود و دا...,1,14.0,13.0


In [12]:
data = data[['comment', 'label_id']]

In [13]:
MODEL_PATH = 'HooshvareLab/bert-fa-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)

In [22]:
input_ids = []
attention_masks = []
for comment in data['comment']:
    encoding = tokenizer.encode_plus(
        comment,
        max_length=64,
        truncation=True,
        add_special_tokens=True,
        return_token_type_ids=True,
        return_attention_mask=True,
        padding='max_length',
    )

    input_ids.append(encoding.get("input_ids"))
    attention_masks.append(encoding.get('attention_mask'))

In [23]:
embedding = np.multiply(input_ids, attention_masks, dtype=float)
embedding[:3]

array([[2.0000e+00, 5.6680e+03, 2.3317e+04, 4.2440e+03, 2.8000e+03,
        2.6283e+04, 4.8750e+03, 8.1130e+03, 4.9570e+03, 2.8710e+03,
        3.3691e+04, 4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.0000e+00, 2.9590e+03, 2.8340e+03, 1.2781e+04, 4.3170e+04,
        3.3620e+03, 3.9730e+03, 3.5510e+03, 8.3530e+03, 2.7

In [24]:
x_train, x_test, y_train, y_test = train_test_split(embedding, data['label_id'], test_size=0.2)

In [26]:
rfc = RandomForestClassifier(criterion='entropy')

rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)
print(classification_report(y_test, y_pred, target_names=['HAPPY', 'SAD']))

              precision    recall  f1-score   support

       HAPPY       0.71      0.62      0.66      6771
         SAD       0.65      0.74      0.69      6606

    accuracy                           0.68     13377
   macro avg       0.68      0.68      0.68     13377
weighted avg       0.68      0.68      0.68     13377

